In [119]:
from lusidtools.jupyter_tools import toggle_code

"""Running a Fund with Investors

Demonstration of how to manage a fund's subscriptions and capital calls with investors in LUSID

Attributes
----------
instruments
transactions
transaction configuration
properties
holdings
"""

toggle_code("Toggle Docstring")

# Running a Fund with Investors

# Set Up

### Import the libraries

In [120]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import json
import lusid
import uuid
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")


print ('LUSID Environment Initialised')
print ('LUSID API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.6.11816.0


![Init](img/fundoperations-lusidinit.gif)

### Create a scope

Here you can create a unique id for your scope to use to hold your fund and investor information.

*Run the cell below to create an identifier for your fund operations scope*

In [121]:
# Generate a unique id for your scope
scope_id = import_data.create_scope_id()
# Set the name of your scope
scope = 'fund-operations-{}'.format(scope_id)
prettyprint.heading('Scope', scope)

Scope: fund-operations-3c29-7a6c-54cb-ce


![Scope](img/fundoperations-scope.gif)

### Create properties to hold important information

As part of your fund operations there are a number of important pieces of information which need to be included on your transactions which are not tracked by LUSID by default. These are:

- SubscriptionType: The type of subscription from an investor into a fund e.g. Committed or Capital
- InvestorId: The identifier for the investor associated with the transaction
- FundCode: The identifier for the fund code associated with the transacton
- OperationStage: The status of the transaction e.g. Draft, Reviewed, Approved

These can be created inside LUSID as [property definitions](https://support.lusid.com/what-is-a-property-definition). 

For further usage of the create property definition API call refer to the [LUSID API Docs: Create Property Definition](https://www.lusid.com/docs/api/#operation/CreatePropertyDefinition).

*Run the cell below to create definitions for each of the properties*

In [122]:
property_codes = ["SubscriptionType", "InvestorId", "FundCode", "OperationStage"]
property_keys = {}

for property_code in property_codes:
    
    # Create our request to define a new property
    request = models.CreatePropertyDefinitionRequest(
        domain='Transaction',
        scope=scope,
        code=property_code,
        value_required=False,
        display_name=property_code,
        data_type_id=models.ResourceId(scope='system', code='string'))

    # Call LUSID to create our new property
    response = api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
        create_property_definition_request=request)

    # Grab the key off the response to use when referencing this property in other LUSID calls
    property_keys[property_code] = response.key

    # Pretty print our key
    prettyprint.heading(f'{property_code} Property Key', response.key)

SubscriptionType Property Key: Transaction/fund-operations-3c29-7a6c-54cb-ce/SubscriptionType
InvestorId Property Key: Transaction/fund-operations-3c29-7a6c-54cb-ce/InvestorId
FundCode Property Key: Transaction/fund-operations-3c29-7a6c-54cb-ce/FundCode
OperationStage Property Key: Transaction/fund-operations-3c29-7a6c-54cb-ce/OperationStage


![Properties](img/fundoperations-properties.gif)

### Create the investor's portfolio

To hold information about your investor's account you need to create a portfolio for each of them.

You start with a single investor which you call "Investor1".

You can read more about creating transaction portfolios in the [LUSID API documentation: LUSID API Docs: Creating Transaction Portfolios](https://www.lusid.com/docs/api/#operation/CreatePortfolio).

*Run the cell below to create a portfolio for your investor*

In [123]:
# Set the code of your portfolio
investor_portfolio_code = 'Investor1'

# Set the creation date of your portfolio 
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=1052)).isoformat()

# Build your request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name=investor_portfolio_code,
    code=investor_portfolio_code,
    base_currency='AUD',
    description=None,
    created=portfolio_creation_date,
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=[property_keys["SubscriptionType"], property_keys["FundCode"]],
    properties=None)

# Call LUSID to create your portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
    scope=scope,
    create_transaction_portfolio_request=request)

# Pretty print the response
prettyprint.portfolio_response(response)

Portfolio Created
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Investor1
Portfolio Effective From: 2020-10-12 15:47:19.349248+00:00
Portfolio Created On: 2023-08-30 15:47:20.517351+00:00



![Investor1](img/fundoperations-investor1.gif)

### Create the fund portfolio

You also need a portfolio to hold your funds. You also start with a single fund called the "Pacific-Infrastructure-Fund".

*Run the cell below to create a portfolio for your fund*

In [124]:
# Set the code of your portfolio
fund_portfolio_code = 'Pacfic-Infrastructure-Fund'

# Set the creation date of your portfolio 
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=1052)).isoformat()

# Build your request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name=fund_portfolio_code,
    code=fund_portfolio_code,
    base_currency='AUD',
    description=None,
    created=portfolio_creation_date,
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=[property_keys["SubscriptionType"], property_keys["InvestorId"]],
    properties=None)

# Call LUSID to create your portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
    scope=scope,
    create_transaction_portfolio_request=request)

# Pretty print the response
prettyprint.portfolio_response(response)

Portfolio Created
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Pacfic-Infrastructure-Fund
Portfolio Effective From: 2020-10-12 15:47:19.616419+00:00
Portfolio Created On: 2023-08-30 15:47:20.800701+00:00



![Fund](img/fundoperations-fund.gif)

### Check the default transaction mapping

Before you can add any transactions to LUSID you need to set up a transaction type configuration. This allows the LUSID movement engine to correctly decompose each transaction into its underlying economic movements. Read more about configuring transaction types here [LUSID Knowledge Base: Configuring Transaction Types](https://support.lusid.com/configuring-transaction-types) and movements in LUSID here [LUSID Knowledge Base: Movements](https://support.lusid.com/what-is-a-movement-in-lusid). 

LUSIS is already pre-configured with some commonly used transaction types. 

*Run the cell below to see what is already in the system*

In [125]:
transaction_configuration_api = api_factory.build(lusid.api.TransactionConfigurationApi)

In [126]:
txn_types_response = list(transaction_configuration_api.list_transaction_types().values())
transaction_types = [ txn_type  for txn_type_list in txn_types_response for txn_type in txn_type_list]
aliases = [txn_type.aliases for txn_type in transaction_types]
list_aliases = pd.DataFrame([alias.to_dict() for sublist in aliases for alias in sublist])
list_aliases.head(10)

,type,description,transaction_class,transaction_roles,is_default
0,DI,DIVIDEND INCOME,Dividend,Longer,False
1,DE,DIVIDEND EXPENSE,Dividend,Shorter,False
2,BY,PURCHASE,Basic,LongLonger,False
3,SL,SALE,Basic,LongShorter,False
4,CS,COVER SHORT,Basic,ShortLonger,False
5,SS,SELL SHORT,Basic,ShortShorter,False
6,SI,SECURITY IN,StockTransfers,LongLonger,False
7,SW,SHORT SECURITY OUT,StockTransfers,ShortLonger,False
8,SO,SECURITY OUT,StockTransfers,LongShorter,False
9,SA,SHORT SECURITY IN,StockTransfers,ShortShorter,False


![Default Transaction Mapping](img/fundoperations-transactiontypeconfig.gif)

## Add transaction types for Fund Operations

By default LUSID likely is not familar with the different types of transactions that you have in operating your fund. 

Rather than having to map your transaction types into LUSID transaction types. You can create your own transaction types and define their economic meaning.

You will do this for each of the transaction types below.

### Commitment

A commitment is when an investor commits cash to one of your funds. You don't require this cash just yet, but you want to note that it has been committed by the investor and is available to you upon the issuance of a Capital Call.

You define two transaction types below.

- One for the fund which indicates that you have a commitment from the investor associated with the transaction
- One for the investor which indicates that this investor has a commitment to the fund associated with the transaction

When you have a commitment transaction both of these transaction types will be used.

*Run the cell below to create your two transaction types*

In [127]:
# Add default side definitions to the non-default transaction type scope.
# If working in the default scope, these side definitions are set by default so, unless these sides have been removed, this setting of sides can be skipped.
default_side_definitions = [
    models.SidesDefinitionRequest(
        side="Side1", 
        side_request=models.SideDefinitionRequest(
            security="Txn:LusidInstrumentId",
            currency="Txn:TradeCurrency",
            rate="Txn:TradeToPortfolioRate",
            units="Txn:Units",
            amount="Txn:TradeAmount")),
    models.SidesDefinitionRequest(
        side="Side2", 
        side_request=models.SideDefinitionRequest(
            security="Txn:SettleCcy",
            currency="Txn:SettlementCurrency",
            rate="SettledToPortfolioRate",
            units="Txn:TotalConsideration",
            amount="Txn:TotalConsideration"))
]

transaction_configuration_api.set_side_definitions(default_side_definitions, scope = scope)

# Construct the transaction type requests and insert them
new_transaction_config = models.TransactionTypeRequest(
    aliases=[
        models.TransactionTypeAlias(
            type='Commitment',
            description='The commitment of funds by an investor',
            transaction_class='FundOperations',
            transaction_roles='Longer')
    ],
    movements=[
        models.TransactionTypeMovement(
            movement_types='CashCommitment',
            side='Side2',
            direction=1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Committed")
            ]
        )
    ],
    properties=None)


new_txn_config = transaction_configuration_api.set_transaction_type(
    source="FundOperations-Investor",
    type="Commitment",
    transaction_type_request=new_transaction_config,
    scope=scope
)

new_transaction_config = models.TransactionTypeRequest(
    aliases=[
        models.TransactionTypeAlias(
            type='Commitment',
            description='The commitment of funds by an investor',
            transaction_class='FundOperations',
            transaction_roles='Longer')
    ],
    movements=[
        models.TransactionTypeMovement(
            movement_types='CashCommitment',
            side='Side2',
            direction=-1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Committed")
            ]),
        models.TransactionTypeMovement(
            movement_types='CashCommitment',
            side='Side2',
            direction=1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Committed"),
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["InvestorId"],
                    set_to="")
            ]
        )
    ],
    properties=None)

new_txn_config = transaction_configuration_api.set_transaction_type(
    source="FundOperations-Fund",
    type="Commitment",
    transaction_type_request=new_transaction_config,
    scope=scope
)

# Call LUSID to update the transaction type scope of both portfolios
patch_document = [
    {
        "value": scope,
        "path": "/transactiontypescope",
        "op": "add"
    }
]
portfolio_codes = [investor_portfolio_code, fund_portfolio_code]

for portfolio_code in portfolio_codes:
    patch_response = api_factory.build(lusid.api.TransactionPortfoliosApi).patch_portfolio_details(
        scope=scope,
        code=portfolio_code,
        operation=patch_document)
    prettyprint.portfolio_response(patch_response)

Portfolio Details: 
Detail Origin Portfolio Scope: fund-operations-3c29-7a6c-54cb-ce
Detail Origin Portfolio Code: Investor1
Base Currency: AUD
Accounting Method: AverageCost
Corporate Action Source Id: None


Portfolio Details: 
Detail Origin Portfolio Scope: fund-operations-3c29-7a6c-54cb-ce
Detail Origin Portfolio Code: Pacfic-Infrastructure-Fund
Base Currency: AUD
Accounting Method: AverageCost
Corporate Action Source Id: None




![Add Transaction Mapping](img/fundoperations-addtype.gif)

### Capital call

A capital call is when you ask your investors to provide the capital that they have previously committed to the fund associated with the capital call.

You define two transaction types below.

- One for the fund which moves the committed balance for the investor associated with the capital call to a capital balance which can be used for transactions
- One for the investor which moves the committed balance for the fund associated with the capital call to a capital balance which has been provided to the fund to use for transactions

When you issue a capital call both of these transaction types will be used.

*Run the cell below to create your two transaction types*

In [128]:
new_transaction_config = models.TransactionTypeRequest(
    aliases=[
        models.TransactionTypeAlias(
            type='CapitalCall',
            description='The provision of funds by an investor in response to a capital call',
            transaction_class='FundOperations',
            transaction_roles='LongShorter')
    ],
    movements=[
        models.TransactionTypeMovement(
            movement_types='CashSettlement',
            side='Side2',
            direction=-1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Committed")
            ]),
        models.TransactionTypeMovement(
            movement_types='CashCommitment',
            side='Side2',
            direction=1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Capital")
            ]
        )
    ],
    properties=None)

new_txn_config = api_factory.build(lusid.api.TransactionConfigurationApi).set_transaction_type(
    source="FundOperations-Investor",
    type="CapitalCall",
    transaction_type_request=new_transaction_config,
    scope=scope
)

new_transaction_config = models.TransactionTypeRequest(
    aliases=[
        models.TransactionTypeAlias(
            type='CapitalCall',
            description='The provision of funds by an investor in response to a capital call',
            transaction_class='FundOperations',
            transaction_roles='Longer')
    ],
    movements=[
        models.TransactionTypeMovement(
            movement_types='CashSettlement',
            side='Side2',
            direction=1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Committed")
            ]),
         models.TransactionTypeMovement(
            movement_types='CashSettlement',
            side='Side2',
            direction=-1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Capital")
            ]),
        models.TransactionTypeMovement(
            movement_types='CashSettlement',
            side='Side2',
            direction=-1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Committed"),
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["InvestorId"],
                    set_to="")
            ]),
        models.TransactionTypeMovement(
            movement_types='CashSettlement',
            side='Side2',
            direction=1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Capital"),
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["InvestorId"],
                    set_to="")
            ]
        )
    ],
    properties=None)

new_txn_config = transaction_configuration_api.set_transaction_type(
    source="FundOperations-Fund",
    type="CapitalCall",
    transaction_type_request=new_transaction_config,
    scope=scope
)

### Distribution

A distribution is when income is distributed from the fund to investors. 

You define two transaction types below.

- One for the fund which generates a distribution of income against the investor associated with the distribution
- One for the investor which receives a distribution of income from to fund associated with the distribution

When you have a distribution event both of these transaction types will be used.

*Run the cell below to create your two transaction types*

In [129]:
# Call LUSID to create your transaction type

new_transaction_config = models.TransactionTypeRequest(
    aliases=[
        models.TransactionTypeAlias(
            type='Distribution',
            description='Income distributed from a fund',
            transaction_class='FundOperations',
            transaction_roles='Longer')
    ],
    movements=[
        models.TransactionTypeMovement(
            movement_types='CashCommitment',
            side='Side2',
            direction=1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Income")
            ]
        )
    ],
    properties=None)

new_txn_config = transaction_configuration_api.set_transaction_type(
    source="FundOperations-Investor",
    type="Distribution",
    transaction_type_request=new_transaction_config,
    scope=scope
)

new_transaction_config = models.TransactionTypeRequest(
    aliases=[
        models.TransactionTypeAlias(
            type='Distribution',
            description='Income distributed from a fund',
            transaction_class='FundOperations',
            transaction_roles='Shorter')
    ],
    movements=[
        models.TransactionTypeMovement(
            movement_types='CashCommitment',
            side='Side2',
            direction=-1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Income")
            ]
        )
    ],
    properties=None)

new_txn_config = transaction_configuration_api.set_transaction_type(
    source="FundOperations-Fund",
    type="Distribution",
    transaction_type_request=new_transaction_config,
    scope=scope
)

## Add transaction types for Asset Transactions

### Initial investment

An initial investment in an asset.

This is only a single transaction type which reduces the amount of capital cash and increases the amount of the asset held in the fund associated with the transaction.

*Run the cell below to create your transaction type*

In [130]:
new_transaction_config = models.TransactionTypeRequest(
    aliases=[
        models.TransactionTypeAlias(
            type='InitialInvestment',
            description='The purchase of a security',
            transaction_class='AssetTransactions',
            transaction_roles='Longer')
    ],
    movements=[
        models.TransactionTypeMovement(
            movement_types='StockMovement',
            side='Side1',
            direction=1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["InvestorId"],
                    set_to=""),
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Capital")
            ]
        ),
        models.TransactionTypeMovement(
            movement_types='CashCommitment',
            side='Side2',
            direction=-1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Capital"),
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["InvestorId"],
                    set_to="")
            ]
        )
    ],
    properties=None)

new_txn_config = api_factory.build(lusid.api.TransactionConfigurationApi).set_transaction_type(
    source="AssetTransactions",
    type="InitialInvestment",
    transaction_type_request=new_transaction_config,
    scope=scope
)

### Distribution

A distribution from an asset.

This is only a single transaction type which increases the amount of capital cash held in the fund associated with the transaction.

*Run the cell below to create the transaction*

In [131]:
new_transaction_config = models.TransactionTypeRequest(
    aliases=[
        models.TransactionTypeAlias(
            type='Distribution',
            description='Income distributed from an infrastructure or real asset',
            transaction_class='AssetTransactions',
            transaction_roles='Longer')
    ],
    movements=[
        models.TransactionTypeMovement(
            movement_types='CashCommitment',
            side='Side2',
            direction=1,
            properties=None,
            mappings=[
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["SubscriptionType"],
                    set_to="Income"),
                models.TransactionTypePropertyMapping(
                    property_key=property_keys["InvestorId"],
                    set_to="")
            ]
        )
    ],
    properties=None)

new_txn_config = transaction_configuration_api.set_transaction_type(
    source="AssetTransactions",
    type="Distribution",
    transaction_type_request=new_transaction_config,
    scope=scope
)

![Final Transaction Mapping](img/fundoperations-finaltypes.gif)

### Set up dates to work with

In [132]:
current_time = datetime.now(pytz.UTC)
day_1 = current_time - timedelta(days=6)
day_2 = current_time - timedelta(days=5)
day_3 = current_time - timedelta(days=4)
day_4 = current_time - timedelta(days=3)
day_5 = current_time - timedelta(days=2)
day_6 = current_time - timedelta(days=1)

### Create a function to make fund operation transactions

In [133]:
def subscription_transaction(date, units, transaction_type, investor_portfolio_code, fund_portfolio_code, status=None):
    """
    This function creates subscription transactions for committing to a fund as well as provisioning capital via
    a capital call. 
    
    param date (datetime): The datetime to upsert the transactions at
    param units (float): The number of units in the transaction
    param transaction_type (str): The code for the transaction type for this subscription transaction
    param investor_portfolio_code (str): The code for the investor's portfolio
    param fund_portfolio_code (str): The code for the fund portfolio
    
    return unique_transaction_id (str): The unique id for the transactions
    """
    
    transactions = {}
    
    unique_transaction_id = str(uuid.uuid4())
    
    investor_properties = {
        property_keys['FundCode']: models.ModelProperty(
            key=property_keys['FundCode'],
            value=models.PropertyValue(
                label_value=fund_portfolio_code)
        )
    }
    
    if type(status) is not type(None):
        investor_properties[property_keys['OperationStage']] = models.ModelProperty(
            key=property_keys['OperationStage'],
            value=models.PropertyValue(
                label_value=status)
        )
            
            
    # Construct the transaction for the investor's portfolio
    transactions[investor_portfolio_code] = models.TransactionRequest(
        transaction_id=unique_transaction_id,
        type=transaction_type,
        instrument_identifiers={
            'Instrument/default/Currency': 'AUD'
        },
        transaction_date=date.isoformat(),
        settlement_date=date.isoformat(),
        units=units,
        transaction_price=models.TransactionPrice(
            price=1,
            type='Price'),
        total_consideration=models.CurrencyAndAmount(
            amount=units,
            currency='AUD'),
        source='FundOperations-Investor',
        transaction_currency='AUD',
        properties=investor_properties)
    
    # Construct the transaction for the fund's portfolio
    transactions[fund_portfolio_code] = models.TransactionRequest(
        transaction_id=unique_transaction_id,
        type=transaction_type,
        instrument_identifiers={
            'Instrument/default/Currency': 'AUD'
        },
        transaction_date=date.isoformat(),
        settlement_date=date.isoformat(),
        units=units,
        transaction_price=models.TransactionPrice(
            price=1,
            type='Price'),
        total_consideration=models.CurrencyAndAmount(
            amount=units,
            currency='AUD'),
        source='FundOperations-Fund',
        transaction_currency='AUD',
        properties={
            property_keys['InvestorId'] : models.ModelProperty(
                key=property_keys['InvestorId'],
                value=models.PropertyValue(
                    label_value=investor_portfolio_code)
            )
        })

    # Loop over the portfolios
    for portfolio_code, transaction in transactions.items():
        
        # Call LUSID to upsert the transactions
        response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
            scope=scope,
            code=portfolio_code,
            transaction_request=[transaction])

        # Print the response from LUSID using pretty formatting 
        prettyprint.transactions_response(
            response,
            scope,
            portfolio_code)
    
    return unique_transaction_id

# Day 1 - Investor commits AUD10,000,000 to the fund

### Make the commitment transaction

"Investor1" commits AUD$10,000,000 to the "Pacfic-Infrastructure-Fund" fund.

Because of the transaction types you configured earlier this is logged as a commitment to the "Pacfic-Infrastructure-Fund" in Investor1's portfolio and a commitment from "Investor1" in the Pacific-Infrastructure-Fund portfolio.

*Run the cell below to make the commitment transaction from Investor1 for the Pacific-Infrastructure-Fund*

In [134]:
subscription_transaction(
    date=day_1, 
    units=10000000, 
    transaction_type='Commitment',
    investor_portfolio_code=investor_portfolio_code,
    fund_portfolio_code=fund_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Investor1
Transactions Effective From: 2023-08-24 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:23.631012+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2023-08-24 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:24.061207+00:00



'e536e64a-3700-4db3-a90b-a30f0c225a67'

![Comittment Transaction](img/fundoperations-day1commitment.gif)

### Get the holdings of the investor's portfolio

Here you can see that Investor1 has a commitment logged against the Pacific-Infrastructure-Fund in their account.

*Run the cell below to generate the holdings for Investor1*

In [135]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/FundCode,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,10000000.00,Pacfic-Infrastructure-Fund,Committed,AUD,Investor1,fund-operations-3c29-7a6c-54cb-ce,10000000.00,AUD


![Comittment Holdings](img/fundoperations-day1holding.gif)

### Get the holdings of the fund portfolio

Here you can see that the Pacific-Infrastructure-Fund has a commitment asset and a commitment liability logged against Investor1.

*Run the cell below to generate the holdings for the Pacific-Infrastructure-Fund*

In [136]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/InvestorId,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,-10000000.00,Investor1,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-10000000.00,AUD
1,CCY_AUD,10000000.00,,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,10000000.00,AUD


# Day 2 - The fund makes a capital call of AUD5,000,000 to the investor

### The investor responds to the capital call and provides the funds

Here the Pacific-Infrastructure-Fund issue a capital call which Investor1 responds to. 

This has the effect of moving the commitment balance to being a capital balance across both Investor1's account and the fund's account.

*Run the cell below to issue the capital call*

In [137]:
subscription_transaction(
    date=day_2, 
    units=5000000, 
    transaction_type='CapitalCall',
    investor_portfolio_code=investor_portfolio_code,
    fund_portfolio_code=fund_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Investor1
Transactions Effective From: 2023-08-25 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:25.172752+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2023-08-25 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:25.538801+00:00



'f55ecbfe-13e8-4a65-9746-48c620396b9e'

### See the updated investor's holdings

Here you can see the updated holdings of the investor in which AUD$5,000,000 has moved from being committed to being capital.

*Run the cell below to generate Investor1's updated holdings after the capital call*

In [138]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/FundCode,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,5000000.00,Pacfic-Infrastructure-Fund,Committed,AUD,Investor1,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD
1,CCY_AUD,5000000.00,Pacfic-Infrastructure-Fund,Capital,AUD,Investor1,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD


![Capital Call Holdings](img/fundoperations-day2holding.gif)

### The fund tracks this response

Here you can see the updated holdings of the Pacific-Infrastructure-Fund in which AUD$5,000,000 has moved from being committed to being capital.

*Run the cell below to generate the funds's updated holdings after the capital call*

In [139]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/InvestorId,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,-5000000.00,Investor1,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-5000000.00,AUD
1,CCY_AUD,5000000.00,,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD
2,CCY_AUD,-5000000.00,Investor1,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-5000000.00,AUD
3,CCY_AUD,5000000.00,,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD


# Day 3 - The fund purchases an alternative physical asset

### Add the alternative asset to the LUSID instrument master

You want to purchase a shopping centre for the Pacific Infrastructure Fund. Although you usually invest in ports and power stations, this investment was just too good for you to pass up.

Before you can log a transaction against this asset you need to add it to your security master.

*Run the cell below to add the shopping centre to your security master*

In [140]:
# Create the bespoke hedge contract definition
shopping_centre_asset = {
    "identifier": "SYD_241232",
    "no_units": 76,
    "average_retail_lease": "2.3 years",
    "term": "5yr",
    "freq": "Quarterly",
    "rating": "B1"
}

# Create the definition for your instrument, attaching the bespoke contract
shopping_centre_asset_instrument = models.InstrumentDefinition(
    name='Sydney_Bondi_Junc_Westfield',
    identifiers={
        'ClientInternal': models.InstrumentIdValue(
            value=shopping_centre_asset["identifier"])},
    definition=models.ExoticInstrument(
        instrument_format=models.InstrumentDefinitionFormat(
        source_system='ClientSystemA',
        vendor='ClientA',
        version='0.0.1'
        ),
        content=json.dumps(shopping_centre_asset),
        instrument_type='ExoticInstrument'            
    )    
)

response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    request_body={"shopping_centre": shopping_centre_asset_instrument})

prettyprint.instrument_response(response)

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,shopping_centre,SYD_241232,LUID_00003EKS


### Purchase the asset

You can now purchase the asset using the capital you have raised from Investor1.

*Run the cell below to purchase the asset*

In [141]:
transaction = models.TransactionRequest(
    transaction_id='Purchase_22319041341',
    type='InitialInvestment',
    instrument_identifiers={
        'Instrument/default/ClientInternal': 'SYD_241232'
    },
    transaction_date=day_3.isoformat(),
    settlement_date=day_3.isoformat(),
    units=1,
    transaction_price=models.TransactionPrice(
        price=5000000,
        type='Price'),
    total_consideration=models.CurrencyAndAmount(
        amount=5000000,
        currency='AUD'),
    source='AssetTransactions',
    transaction_currency='AUD')

# Call LUSID to upsert your transactions for this portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
    scope=scope,
    code=fund_portfolio_code,
    transaction_request=[transaction])

# Print the response from LUSID using pretty formatting 
prettyprint.transactions_response(
    response,
    scope,
    fund_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2023-08-26 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:26.639372+00:00



### See the updated fund holdings

With the asset purchased you can now see it in the fund's updated holdings.

*Run the cell below to get the fund's updated holdings*

In [142]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/InvestorId,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,-5000000.00,Investor1,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-5000000.00,AUD
1,CCY_AUD,5000000.00,,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD
2,CCY_AUD,-5000000.00,Investor1,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-5000000.00,AUD
3,LUID_00003EKS,1.00,,Capital,Sydney_Bondi_Junc_Westfield,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD


# Day 4 - Investor 2 Makes AUD2,500,000 committment 

### Create a portfolio for the new investor

In [143]:
# Set the code of your portfolio
investor2_portfolio_code = 'Investor2'

# Set the creation date of your portfolio 
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=1052)).isoformat()

# Build your request to create your portfolio
request = models.CreateTransactionPortfolioRequest(
    display_name=investor2_portfolio_code,
    code=investor2_portfolio_code,
    base_currency='AUD',
    description=None,
    created=portfolio_creation_date,
    corporate_action_source_id=None,
    accounting_method='AverageCost',
    sub_holding_keys=[property_keys["SubscriptionType"], property_keys["FundCode"], property_keys["OperationStage"]],
    properties=None,
    transaction_type_scope=scope)

# Call LUSID to create your portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
    scope=scope,
    create_transaction_portfolio_request=request)

# Pretty print the response
prettyprint.portfolio_response(response)

Portfolio Created
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Investor2
Portfolio Effective From: 2020-10-12 15:47:25.987513+00:00
Portfolio Created On: 2023-08-30 15:47:27.153282+00:00



### Create a commitment for the investor

Investor2 commits AUD$2,500,000 to the Pacific-Infrastructure-Fund. 

Unlike Investor1 his commitment is not automatically approved and is marked as being in DRAFT status.

*Run the cell below to create the draft commitment*

In [144]:
transaction_id = subscription_transaction(
    date=day_4, 
    units=2500000, 
    transaction_type='Commitment',
    investor_portfolio_code=investor2_portfolio_code,
    fund_portfolio_code=fund_portfolio_code,
    status='Draft')

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Investor2
Transactions Effective From: 2023-08-27 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:27.591516+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2023-08-27 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:27.963853+00:00



You can see the Draft commitment by looking at the holdings of Investor2's account.

*Run the cell below to see Investor2's holdings*

In [145]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor2_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/FundCode,Transaction/OperationStage,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,2500000.00,Pacfic-Infrastructure-Fund,Draft,Committed,AUD,Investor2,fund-operations-3c29-7a6c-54cb-ce,2500000.00,AUD


## Investor2's commitment is flagged for review

The commitment by Investor2 is flagged for review, while this review process is ongoing the status of the commitment is updated.

*Run the cell below to update the commitment status to "Review"*

In [146]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transaction_properties(
    scope=scope,
    code=investor2_portfolio_code,
    transaction_id=transaction_id,
    request_body={property_keys['OperationStage']: models.ModelProperty(
        key=property_keys['OperationStage'],
        value=models.PropertyValue(
            label_value='Review')
    )})

prettyprint.add_transaction_property(response)

Added transaction properties asAt: 2023-08-30 15:47:28.548321+00:00


You can see the commitment has been moved to "Review" status by looking at the holdings of Investor2's account.

*Run the cell below to see Investor2's holdings*

In [147]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor2_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/FundCode,Transaction/OperationStage,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,2500000.00,Pacfic-Infrastructure-Fund,Review,Committed,AUD,Investor2,fund-operations-3c29-7a6c-54cb-ce,2500000.00,AUD


Finally the commitment by Investor2 is approved.

*Run the cell below to update the commitment status to "Approved"*

In [148]:
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transaction_properties(
    scope=scope,
    code=investor2_portfolio_code,
    transaction_id=transaction_id,
    request_body={property_keys['OperationStage']: models.ModelProperty(
        key=property_keys['OperationStage'],
        value=models.PropertyValue(
            label_value='Approved')
    )})

prettyprint.add_transaction_property(response)

Added transaction properties asAt: 2023-08-30 15:47:29.188186+00:00


You can see the commitment has been moved to "Approved" status by looking at the holdings of Investor2's account.

*Run the cell below to see Investor2's holdings*

In [149]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor2_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/FundCode,Transaction/OperationStage,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,2500000.00,Pacfic-Infrastructure-Fund,Approved,Committed,AUD,Investor2,fund-operations-3c29-7a6c-54cb-ce,2500000.00,AUD


### Issue a capital call

You now issue a second capital call. This time it is for AUD$2,500,000 to Investor2 for the Pacific-Infrastructure-Fund.

*Run the cell below to issue the second capital call*

In [150]:
subscription_transaction(
    date=day_4, 
    units=2500000, 
    transaction_type='CapitalCall',
    investor_portfolio_code=investor2_portfolio_code,
    fund_portfolio_code=fund_portfolio_code,
    status='Approved')

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Investor2
Transactions Effective From: 2023-08-27 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:30.026859+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2023-08-27 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:30.383820+00:00



'f2f54431-1961-4381-980c-8c1872b928e8'

### See the updated fund holdings

You can see now in the fund's holdings capital from both Investor1 and Investor2.

*Run the cell below to generate the updated holdings for the fund*

In [151]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/InvestorId,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,-5000000.00,Investor1,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-5000000.00,AUD
1,CCY_AUD,5000000.00,,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD
2,CCY_AUD,-5000000.00,Investor1,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-5000000.00,AUD
3,CCY_AUD,2500000.00,,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,2500000.00,AUD
4,CCY_AUD,-2500000.00,Investor2,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-2500000.00,AUD
5,LUID_00003EKS,1.00,,Capital,Sydney_Bondi_Junc_Westfield,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD


# Day 5 - Income is generated from the asset

### Log the income transaction

The shopping centre returns income of AUD$50,000 via rental income.

*Run the cell below to generate the rental income*

In [152]:
transaction = models.TransactionRequest(
    transaction_id='Income_22319041341',
    type='Distribution',
    instrument_identifiers={
        'Instrument/default/ClientInternal': 'SYD_241232'
    },
    transaction_date=day_5.isoformat(),
    settlement_date=day_5.isoformat(),
    units=1,
    transaction_price=models.TransactionPrice(
        price=50000,
        type='Price'),
    total_consideration=models.CurrencyAndAmount(
        amount=50000,
        currency='AUD'),
    source='AssetTransactions',
    transaction_currency='AUD',
    properties={property_keys['OperationStage']: models.ModelProperty(
        key=property_keys['OperationStage'],
        value=models.PropertyValue(
            label_value='Approved')
    )})

# Call LUSID to upsert your transactions for this portfolio
response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(
    scope=scope,
    code=fund_portfolio_code,
    transaction_request=[transaction])

# Print the response from LUSID using pretty formatting 
prettyprint.transactions_response(
    response,
    scope,
    fund_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2023-08-28 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:31.090333+00:00



### Get the updated fund holdings

You can see this income in the fund where it has not yet been allocated to any investors.

*Run the cell below to get the updated fund holdings*

In [153]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/InvestorId,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,-5000000.00,Investor1,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-5000000.00,AUD
1,CCY_AUD,5000000.00,,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD
2,CCY_AUD,-5000000.00,Investor1,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-5000000.00,AUD
3,CCY_AUD,2500000.00,,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,2500000.00,AUD
4,CCY_AUD,-2500000.00,Investor2,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-2500000.00,AUD
5,CCY_AUD,50000.00,,Income,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,50000.00,AUD
6,LUID_00003EKS,1.00,,Capital,Sydney_Bondi_Junc_Westfield,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD


# Day 6 - Income is distributed to investors

You decide to distribute the recent income from the shopping centre to the two investors, who you (perhaps unfairly and in error) award an equal share of the proceeds.

*Run the cell below to distribute the income*

In [154]:
subscription_transaction(
    date=day_6, 
    units=25000, 
    transaction_type='Distribution',
    investor_portfolio_code=investor_portfolio_code,
    fund_portfolio_code=fund_portfolio_code,
    status='Approved')

subscription_transaction(
    date=day_6, 
    units=25000, 
    transaction_type='Distribution',
    investor_portfolio_code=investor2_portfolio_code,
    fund_portfolio_code=fund_portfolio_code,
    status='Approved')

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Investor1
Transactions Effective From: 2023-08-29 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:31.778403+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2023-08-29 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:32.140841+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Investor2
Transactions Effective From: 2023-08-29 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:32.526496+00:00

Transactions Successfully Upserted into Portfolio
Scope: fund-operations-3c29-7a6c-54cb-ce
Code: Pacfic-Infrastructure-Fund
Transactions Effective From: 2023-08-29 15:47:21.979004+00:00
Transactions Created On: 2023-08-30 15:47:32.876045+00:00



'ab293c43-7eb5-4d1c-b98b-01b8fed71723'

You can now see the update fund holdings representing the income being transferred to the investors.

*Run the cell below to see the updated fund holdings*

In [155]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=fund_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/InvestorId,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,-5000000.00,Investor1,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-5000000.00,AUD
1,CCY_AUD,5000000.00,,Committed,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD
2,CCY_AUD,-5000000.00,Investor1,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-5000000.00,AUD
3,CCY_AUD,2500000.00,,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,2500000.00,AUD
4,CCY_AUD,-2500000.00,Investor2,Capital,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-2500000.00,AUD
5,CCY_AUD,50000.00,,Income,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,50000.00,AUD
6,CCY_AUD,-25000.00,Investor1,Income,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-25000.00,AUD
7,CCY_AUD,-25000.00,Investor2,Income,AUD,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,-25000.00,AUD
8,LUID_00003EKS,1.00,,Capital,Sydney_Bondi_Junc_Westfield,Pacfic-Infrastructure-Fund,fund-operations-3c29-7a6c-54cb-ce,5000000.00,AUD


You can see this income now appear in both investors accounts.

*Run the cell below to see the income in Investor2's account*

In [156]:
holdings = api_factory.build(lusid.api.TransactionPortfoliosApi).get_holdings(
    scope=scope,
    code=investor2_portfolio_code,
    property_keys=['Instrument/default/Name'])

prettyprint.sub_holdings(holdings)

,Lusid Unique Instrument Id,Units,Transaction/FundCode,Transaction/OperationStage,Transaction/SubscriptionType,Instrument_Name,Holding_SourcePortfolioId,Holding_SourcePortfolioScope,Total Cost,Currency
0,CCY_AUD,2500000.00,Pacfic-Infrastructure-Fund,Approved,Capital,AUD,Investor2,fund-operations-3c29-7a6c-54cb-ce,2500000.00,AUD
1,CCY_AUD,25000.00,Pacfic-Infrastructure-Fund,Approved,Income,AUD,Investor2,fund-operations-3c29-7a6c-54cb-ce,25000.00,AUD
